In [ ]:
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pickle
import nltk

# Load intents data from the JSON file
with open('intents-sql.json') as file:
    intents = json.load(file)

# Initialize the lemmatizer and tokenizer
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Preprocess data: tokenize and lemmatize the patterns
training_sentences = []
training_labels = []
classes = []
responses = {}

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern.lower())
        training_sentences.append(' '.join([lemmatizer.lemmatize(word) for word in word_list]))
        training_labels.append(intent['tag'])
    
    if intent['tag'] not in classes:
        classes.append(intent['tag'])
    
    responses[intent['tag']] = intent['responses']

# Initialize the tokenizer and label encoder
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_sentences)
X_train = tokenizer.texts_to_sequences(training_sentences)

# Set maxlen based on the length of the longest sentence
max_len = max([len(sentence.split()) for sentence in training_sentences]) 
X_train = pad_sequences(X_train, padding='post', maxlen=max_len) 

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(training_labels)

# Save tokenizer and label encoder
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
    
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle)

# Reshape data for LSTM input (3D)
X_train = np.expand_dims(X_train, -1)

# Create the model with increased complexity
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.6))  # Increased dropout rate
model.add(Dense(len(classes), activation='softmax'))  # Output layer

# Compile the model with a learning rate scheduler
optimizer = Adam(learning_rate=0.001)  # Increase the learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up the learning rate scheduler to reduce LR when the accuracy plateaus
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=0.00001)

# Train the model with a larger batch size
model.fit(X_train, y_train, epochs=300, batch_size=8, callbacks=[lr_scheduler])

# Save the trained model
model.save('chatbot_model.keras')

print("Training complete and model saved!")



In [ ]:
# El dataset utilizado en este código es el contenido del archivo intents-sql.json. Dependiendo del chatbot que se esté utilizando se utilizará un dataset u otro, pero el modelos de entrenamiento es similar. Todos los datasets siguen un formato estándar para entrenar chatbots basados en aprendizaje profundo, y generalmente contiene una lista de intenciones (intents) estructuradas como un JSON con las siguientes claves:

# patterns: Son frases o expresiones de entrada que los usuarios podrían escribir. Estas se usan como los datos de entrada (features) del modelo.
# tag: Es la etiqueta asociada a cada intención, que actúa como la clase de salida (label) que el modelo debe predecir.
# responses: Son las posibles respuestas que el chatbot puede devolver al usuario si identifica correctamente la intención.

# Los resultados han sido batante satisfactorios, hemos conseguido que los chatbots den respuestas con una precisión aceptable. Sin embargo, siempre hay margen de mejora, y se podrían probar diferentes arquitecturas de modelos, hiperparámetros y técnicas de preprocesamiento de texto para mejorar aún más la precisión del chatbot.

# A continuación mostramos tres ejemplos de los 10 diferentes datasets que hemos utilizado.


In [ ]:
# intents_softskills.json
{
  "intents": [
      {
      "tag": "greetings",
      "patterns": [
        "Hello",
        "Hi",
        "Hey",
        "Good morning",
        "Good afternoon",
        "Good evening",
        "What's up?",
        "How are you?"
        ],
      "responses": [
        "Hello! How can I assist you today?",
        "Hi there! How can I help you?",
        "Hey! What's up?",
        "Good day! How can I help?",
        "Hello! How's it going?"
        ]
      },
      {
      "tag": "goodbye",
      "patterns": [
        "Goodbye",
        "Bye",
        "See you later",
        "Take care",
        "Catch you later",
        "Bye for now"
        ],
      "responses": [
        "Goodbye! Have a great day!",
        "See you later! Take care!",
        "Bye! Hope to chat with you again soon!",
        "Catch you later! Stay safe!"
        ]
      },
      {
      "tag": "thanks",
      "patterns": [
        "Thank you",
        "Thanks",
        "Thanks a lot",
        "I appreciate it",
        "Thanks for your help",
        "Thank you very much"
        ],
      "responses": [
        "You're welcome!",
        "Glad to help!",
        "You're very welcome!",
        "No problem at all!",
        "Happy to help!"
        ]
      },
      {
      "tag": "sorry",
      "patterns": [
        "Sorry",
        "I'm sorry",
        "Apologies",
        "My apologies",
        "I apologize"
        ],
      "responses": [
        "No worries! It's all good.",
        "It's okay! Don't worry about it.",
        "No problem, all forgiven.",
        "It's all right, don't worry."
        ]
      },
      {
      "tag": "affirmative",
      "patterns": [
        "Yes",
        "Yeah",
        "Of course",
        "Sure",
        "Definitely",
        "Absolutely"
        ],
      "responses": [
        "Great! How can I assist you further?",
        "Awesome! What else can I help you with?",
        "Thanks for confirming! What's next?",
        "Wonderful! What's on your mind?"
        ]
      },
      {
      "tag": "negative",
      "patterns": [
        "No",
        "Nope",
        "Not really",
        "I don't think so",
        "Definitely not",
        "Not at all"
        ],
      "responses": [
        "Okay, let me know if you change your mind.",
        "Alright, feel free to ask me anything else!",
        "No problem, let me know if you need anything.",
        "Got it! I'm here if you need anything later."
        ]
      },
    {
      "tag": "dependability",
      "patterns": [
        "What does dependability mean?",
        "How can dependability help in a team?",
        "Why is being dependable important?",
        "Can you explain the importance of dependability?",
        "What are the traits of a dependable person?",
        "How do you show dependability?",
        "Why is dependability a key soft skill?",
        "What makes someone dependable?",
        "How can dependability improve work performance?",
        "What are examples of dependability in the workplace?"
      ],
      "responses": [
        "Dependability means being reliable and consistent in your actions.",
        "Being dependable means others can count on you to follow through on tasks.",
        "Dependability helps build trust within teams and increases productivity.",
        "A dependable person ensures that tasks are completed on time and to a high standard.",
        "Dependability is crucial for establishing strong working relationships and reliability."
      ]
    },
    {
      "tag": "teamwork",
      "patterns": [
        "Why is teamwork important?",
        "What are the qualities of a good team member?",
        "How do you work well with others?",
        "Can you give an example of effective teamwork?",
        "What makes a team successful?",
        "How do you contribute to a team?",
        "What does teamwork look like in the workplace?",
        "Why is collaboration important in teamwork?",
        "How do you solve conflicts in a team?",
        "What makes someone a good team player?"
      ],
      "responses": [
        "Teamwork involves working together toward common goals and helping each other.",
        "Good teamwork is built on trust, communication, and collaboration.",
        "A successful team communicates effectively, shares responsibilities, and supports each other.",
        "Teamwork helps leverage diverse skills and perspectives to achieve better results.",
        "Working well with others involves listening, compromising, and being supportive."
      ]
    },
    {
      "tag": "problem-solving",
      "patterns": [
        "How do you solve a problem?",
        "What is problem-solving in the workplace?",
        "Can you give an example of problem-solving?",
        "Why is problem-solving important?",
        "What are the steps in solving a problem?",
        "How do you approach problem-solving?",
        "What skills are needed for problem-solving?",
        "Can you explain critical problem-solving skills?",
        "What makes a good problem solver?",
        "How do you overcome challenges?"
      ],
      "responses": [
        "Problem-solving involves identifying an issue and finding a solution.",
        "Effective problem-solving requires critical thinking, creativity, and persistence.",
        "A good problem solver breaks down complex issues into manageable parts and seeks solutions.",
        "Problem-solving is important because it helps overcome obstacles and improve performance.",
        "To solve a problem, you should define the issue, explore options, and evaluate outcomes."
      ]
    },
    {
      "tag": "flexibility",
      "patterns": [
        "What is flexibility in the workplace?",
        "How do you demonstrate flexibility?",
        "Why is being flexible important?",
        "Can you provide an example of flexibility?",
        "What does it mean to be adaptable?",
        "How do you deal with unexpected changes?",
        "What are the benefits of being flexible?",
        "How can flexibility improve productivity?",
        "How do you handle change in the workplace?",
        "What makes someone flexible in a team?"
      ],
      "responses": [
        "Flexibility means being able to adjust to new conditions and challenges.",
        "Being flexible allows you to adapt to change and work effectively in different situations.",
        "Flexibility helps you remain calm and productive when things don't go as planned.",
        "A flexible person can quickly change strategies when faced with unexpected challenges.",
        "Adaptability and flexibility are essential for thriving in dynamic work environments."
      ]
    },
    {
      "tag": "critical_thinking",
      "patterns": [
        "What is critical thinking?",
        "Why is critical thinking important?",
        "How can I improve my critical thinking skills?",
        "Can you give an example of critical thinking?",
        "How do you make decisions based on critical thinking?",
        "What does it mean to think critically?",
        "How can critical thinking improve work performance?",
        "What role does critical thinking play in problem-solving?",
        "How does critical thinking affect decision-making?",
        "What are the key aspects of critical thinking?"
      ],
      "responses": [
        "Critical thinking involves analyzing information objectively and making reasoned judgments.",
        "It's about questioning assumptions and considering multiple perspectives before making decisions.",
        "Critical thinking helps you evaluate problems from different angles and find the best solution.",
        "Improving critical thinking helps you make more informed and effective decisions.",
        "Critical thinking is vital for navigating complex situations and finding creative solutions."
      ]
    },
    {
      "tag": "communication",
      "patterns": [
        "What is effective communication?",
        "Why is communication important?",
        "How can I improve my communication skills?",
        "What does good communication look like?",
        "How do you communicate clearly?",
        "What are the barriers to communication?",
        "How do you overcome communication challenges?",
        "What role does communication play in teamwork?",
        "How can communication improve workplace relationships?",
        "Can you give an example of effective communication?"
      ],
      "responses": [
        "Effective communication involves conveying your message clearly and listening actively.",
        "Good communication helps avoid misunderstandings and builds stronger relationships.",
        "Improving communication skills involves being clear, concise, and an active listener.",
        "Clear communication ensures everyone is on the same page and working toward common goals.",
        "Effective communication is key to resolving conflicts and achieving success in teams."
      ]
    },
    {
      "tag": "creativity",
      "patterns": [
        "How do you demonstrate creativity?",
        "What is creativity in the workplace?",
        "Why is creativity important?",
        "Can you give an example of creative thinking?",
        "What are the benefits of creativity?",
        "How does creativity solve problems?",
        "What does it mean to be creative in your work?",
        "How do you foster creativity in a team?",
        "What is creative problem-solving?",
        "Why is creative thinking important for innovation?"
      ],
      "responses": [
        "Creativity involves thinking outside the box to come up with new ideas and solutions.",
        "Being creative in the workplace can lead to innovation and problem-solving.",
        "Creative thinking helps you approach problems in unique and efficient ways.",
        "Fostering creativity means encouraging new ideas, brainstorming, and exploration.",
        "Creativity in work leads to innovative solutions and improved processes."
      ]
    },
    {
      "tag": "cultural_intelligence",
      "patterns": [
        "What is cultural intelligence?",
        "Why is cultural intelligence important?",
        "How can I improve my cultural intelligence?",
        "Can you give an example of cultural intelligence?",
        "What does it mean to have cultural awareness?",
        "How does cultural intelligence benefit teams?",
        "Why is cultural sensitivity important in the workplace?",
        "How do you work with people from different cultures?",
        "What role does cultural intelligence play in communication?",
        "How can cultural intelligence improve work relationships?"
      ],
      "responses": [
        "Cultural intelligence means understanding and respecting cultural differences.",
        "Being culturally intelligent helps you work effectively with people from diverse backgrounds.",
        "Cultural intelligence enhances communication and reduces misunderstandings in multicultural environments.",
        "Improving cultural intelligence involves learning about different cultures and being open-minded.",
        "Cultural intelligence is essential for fostering inclusivity and collaboration in diverse teams."
      ]
    }
  ]
}

In [ ]:
# Intents_html.json
{
  "intents": [
      {
      "tag": "greetings",
      "patterns": [
        "Hello",
        "Hi",
        "Hey",
        "Good morning",
        "Good afternoon",
        "Good evening",
        "What's up?",
        "How are you?"
        ],
      "responses": [
        "Hello! How can I assist you today?",
        "Hi there! How can I help you?",
        "Hey! What's up?",
        "Good day! How can I help?",
        "Hello! How's it going?"
        ]
      },
      {
      "tag": "goodbye",
      "patterns": [
        "Goodbye",
        "Bye",
        "See you later",
        "Take care",
        "Catch you later",
        "Bye for now"
        ],
      "responses": [
        "Goodbye! Have a great day!",
        "See you later! Take care!",
        "Bye! Hope to chat with you again soon!",
        "Catch you later! Stay safe!"
        ]
      },
      {
      "tag": "thanks",
      "patterns": [
        "Thank you",
        "Thanks",
        "Thanks a lot",
        "I appreciate it",
        "Thanks for your help",
        "Thank you very much"
        ],
      "responses": [
        "You're welcome!",
        "Glad to help!",
        "You're very welcome!",
        "No problem at all!",
        "Happy to help!"
        ]
      },
      {
      "tag": "sorry",
      "patterns": [
        "Sorry",
        "I'm sorry",
        "Apologies",
        "My apologies",
        "I apologize"
        ],
      "responses": [
        "No worries! It's all good.",
        "It's okay! Don't worry about it.",
        "No problem, all forgiven.",
        "It's all right, don't worry."
        ]
      },
      {
      "tag": "affirmative",
      "patterns": [
        "Yes",
        "Yeah",
        "Of course",
        "Sure",
        "Definitely",
        "Absolutely"
        ],
      "responses": [
        "Great! How can I assist you further?",
        "Awesome! What else can I help you with?",
        "Thanks for confirming! What's next?",
        "Wonderful! What's on your mind?"
        ]
      },
      {
      "tag": "negative",
      "patterns": [
        "No",
        "Nope",
        "Not really",
        "I don't think so",
        "Definitely not",
        "Not at all"
        ],
      "responses": [
        "Okay, let me know if you change your mind.",
        "Alright, feel free to ask me anything else!",
        "No problem, let me know if you need anything.",
        "Got it! I'm here if you need anything later."
        ]
      },
    {
      "tag": "html setup",
      "patterns": [
        "How do I start an HTML file?",
        "What is the structure of an HTML document?",
        "How can I create a basic HTML page?",
        "How to set up an HTML document?",
        "What tags are needed for an HTML file?",
        "Steps to create an HTML document",
        "HTML document structure",
        "How do I write an HTML file?",
        "What is the basic template for HTML?",
        "How to start coding in HTML?"
      ],
      "responses": [
        "Start with the <!DOCTYPE html> declaration, followed by the <html> tag.",
        "An HTML document begins with <!DOCTYPE html>, then includes <html>, <head>, and <body> tags.",
        "Use <!DOCTYPE html> to define the document type, then add <html>, <head>, and <body>.",
        "An HTML file requires a basic structure with <!DOCTYPE html>, <html>, <head>, and <body> tags.",
        "To set up an HTML document, include a title in <head> and content within <body>.",
        "Start with <!DOCTYPE html> and structure your page using <html>, <head>, and <body> tags."
      ]
    },
    {
      "tag": "html link",
      "patterns": [
        "How do I add a link in HTML?",
        "What is the tag for links in HTML?",
        "How to use the <a> tag?",
        "How do I create hyperlinks in HTML?",
        "Steps to add a link in HTML",
        "What is the syntax for links in HTML?",
        "How to make a clickable link in HTML?",
        "How do I link one page to another in HTML?",
        "HTML link example",
        "How to add URLs to HTML?"
      ],
      "responses": [
        "Use the <a> tag with the href attribute to create a link.",
        "To add a link, write <a href='URL'>Link Text</a>.",
        "The <a> tag is used to define hyperlinks in HTML.",
        "Use <a href='https://example.com'>Click here</a> to create a link.",
        "To link another page, use <a href='page.html'>Page Name</a>.",
        "The href attribute specifies the URL for the link in the <a> tag."
      ]
    },
    {
      "tag": "html image",
      "patterns": [
        "How do I add an image in HTML?",
        "What is the tag for images in HTML?",
        "How to use the <img> tag?",
        "Steps to add an image in HTML",
        "What is the syntax for adding images in HTML?",
        "How to embed an image in HTML?",
        "HTML image example",
        "What attributes are needed for the <img> tag?",
        "How do I display a picture in HTML?",
        "How to include an image in HTML?"
      ],
      "responses": [
        "Use the <img> tag with the src attribute to add an image.",
        "To display an image, write <img src='image.jpg' alt='description'>.",
        "The <img> tag is used to embed images in HTML.",
        "Use <img src='path/to/image.jpg'> to include an image in your HTML.",
        "The src attribute specifies the image path, and alt provides alternative text.",
        "To add an image, include the <img src='image.jpg' alt='description'> tag in your code."
      ]
    },
    {
      "tag": "html table",
      "patterns": [
        "How do I create a table in HTML?",
        "What is the tag for tables in HTML?",
        "How to use the <table> tag?",
        "Steps to add a table in HTML",
        "What is the syntax for tables in HTML?",
        "How to format data in a table using HTML?",
        "HTML table example",
        "What attributes are needed for the <table> tag?",
        "How do I create rows and columns in HTML?",
        "How to include a table in HTML?"
      ],
      "responses": [
        "Use the <table> tag to create a table and <tr>, <td>, and <th> tags to define its structure.",
        "To create a table, write <table> followed by rows with <tr> and cells with <td>.",
        "The <table> tag is used to define tables in HTML.",
        "Use <table> with nested <tr> for rows and <td> for cells to format data.",
        "Add a table header using the <th> tag inside a <tr> row.",
        "The <table> tag includes optional attributes like border, cellpadding, and cellspacing."
      ]
    },
    {
      "tag": "html list",
      "patterns": [
        "How do I create a list in HTML?",
        "What is the tag for lists in HTML?",
        "How to use the <ul> and <ol> tags?",
        "Steps to add a list in HTML",
        "What is the syntax for unordered lists in HTML?",
        "How to create a numbered list in HTML?",
        "HTML list example",
        "What attributes are needed for the <ul> tag?",
        "How do I make a bullet list in HTML?",
        "How to include a list in HTML?"
      ],
      "responses": [
        "Use the <ul> tag for an unordered list and <ol> for an ordered list.",
        "To create a list, write <ul> or <ol> with nested <li> items.",
        "The <ul> tag defines a bullet-point list, while <ol> defines a numbered list.",
        "Use <li> inside <ul> or <ol> to define list items.",
        "An unordered list starts with <ul> and includes <li> items.",
        "A numbered list uses <ol> as the parent tag for <li> items."
      ]
    },
    {
      "tag": "html form",
      "patterns": [
        "How do I create a form in HTML?",
        "What is the tag for forms in HTML?",
        "How to use the <form> tag?",
        "Steps to add a form in HTML",
        "What is the syntax for forms in HTML?",
        "How to include input fields in HTML?",
        "HTML form example",
        "What attributes are needed for the <form> tag?",
        "How do I create a submit button in HTML?",
        "How to handle user input in HTML?"
      ],
      "responses": [
        "Use the <form> tag to create a form and include input elements like <input> and <textarea>.",
        "To create a form, write <form> with input elements and a submit button.",
        "The <form> tag defines a form for collecting user input.",
        "Include input fields using <input type='text'> and buttons with <button>.",
        "The <form> tag can include attributes like action and method for form submission.",
        "Use <form> with child elements like <input> and <button> to collect data."
      ]
    },
    {
      "tag": "html heading",
      "patterns": [
        "How do I create headings in HTML?",
        "What is the tag for headings in HTML?",
        "How to use the <h1> to <h6> tags?",
        "Steps to add headings in HTML",
        "What is the syntax for headings in HTML?",
        "How to structure headings in HTML?",
        "HTML heading example",
        "What are the levels of headings in HTML?",
        "How do I create a title in HTML?",
        "How to format headings in HTML?"
      ],
      "responses": [
        "Use <h1> to <h6> tags to create headings in HTML, with <h1> being the largest.",
        "To add a heading, write <h1> for the main title and smaller tags like <h2>, <h3> for subtitles.",
        "The <h1> to <h6> tags define headings, with decreasing size and importance.",
        "Use <h1> for the main heading and <h2> to <h6> for subheadings.",
        "Headings help structure content, with <h1> being the most prominent and <h6> the least.",
        "Include headings in your HTML using <h1> to <h6> tags as appropriate."
      ]
    },
    {
      "tag": "html paragraph",
      "patterns": [
        "How do I add a paragraph in HTML?",
        "What is the tag for paragraphs in HTML?",
        "How to use the <p> tag?",
        "Steps to add a paragraph in HTML",
        "What is the syntax for paragraphs in HTML?",
        "How to include text in HTML?",
        "HTML paragraph example",
        "What attributes can be used with the <p> tag?",
        "How do I format a paragraph in HTML?",
        "How to add textual content in HTML?"
      ],
      "responses": [
        "Use the <p> tag to create a paragraph of text in HTML.",
        "To add a paragraph, write <p> followed by your text and close with </p>.",
        "The <p> tag defines a paragraph in HTML.",
        "Include text in your HTML document using the <p> tag.",
        "The <p> tag is used for formatting blocks of text in HTML.",
        "Add paragraphs with the <p> tag and close each paragraph with </p>."
      ]
    },
    {
      "tag": "html span",
      "patterns": [
        "How do I use the <span> tag in HTML?",
        "What is the purpose of the <span> tag in HTML?",
        "How to style text using the <span> tag?",
        "Steps to use the <span> tag in HTML",
        "What is the syntax for the <span> tag?",
        "How to apply styles to specific text in HTML?",
        "HTML <span> example",
        "What attributes can be used with the <span> tag?",
        "How do I use <span> for inline text formatting?",
        "How to emphasize part of a sentence in HTML?"
      ],
      "responses": [
        "Use the <span> tag to apply styles or classes to specific inline text.",
        "To style text, wrap it in a <span> tag and apply CSS styles.",
        "The <span> tag is used for inline formatting and applying specific styles.",
        "Include a <span> tag to format or emphasize specific parts of text.",
        "The <span> tag can include attributes like class, id, and style for custom styling.",
        "Use <span> for inline text styling or to target specific text for formatting."
      ]
    },
    {
      "tag": "html div",
      "patterns": [
        "How do I use the <div> tag in HTML?",
        "What is the purpose of the <div> tag in HTML?",
        "How to group elements using the <div> tag?",
        "Steps to use the <div> tag in HTML",
        "What is the syntax for the <div> tag?",
        "How to style sections of a webpage in HTML?",
        "HTML <div> example",
        "What attributes can be used with the <div> tag?",
        "How do I structure a webpage using <div>?",
        "How to create containers in HTML?"
      ],
      "responses": [
        "Use the <div> tag to group and organize sections of a webpage.",
        "The <div> tag is a block-level element used to structure HTML content.",
        "Wrap related content in a <div> tag to apply styles or organize layout.",
        "Include a <div> tag to create containers for HTML elements.",
        "The <div> tag can include attributes like class, id, and style for custom layouts.",
        "Use <div> to group elements and apply consistent styles or layouts to sections."
      ]
    }
  ]
}

In [ ]:
# intents_django.json
{
  "intents": [
      {
          "tag": "greetings",
          "patterns": [
              "Hello",
              "Hi",
              "Hey",
              "Good morning",
              "Good afternoon",
              "Good evening",
              "What's up?",
              "How are you?"
        ],
          "responses": [
              "Hello! How can I assist you today?",
              "Hi there! How can I help you?",
              "Hey! What's up?",
              "Good day! How can I help?",
              "Hello! How's it going?"
        ]
      },
      {
          "tag": "goodbye",
          "patterns": [
              "Goodbye",
              "Bye",
              "See you later",
              "Take care",
              "Catch you later",
              "Bye for now"
        ],
          "responses": [
              "Goodbye! Have a great day!",
              "See you later! Take care!",
              "Bye! Hope to chat with you again soon!",
              "Catch you later! Stay safe!"
        ]
      },
      {
          "tag": "thanks",
          "patterns": [
              "Thank you",
              "Thanks",
              "Thanks a lot",
              "I appreciate it",
              "Thanks for your help",
              "Thank you very much"
        ],
          "responses": [
              "You're welcome!",
              "Glad to help!",
              "You're very welcome!",
              "No problem at all!",
              "Happy to help!"
        ]
      },
      {
          "tag": "sorry",
          "patterns": [
              "Sorry",
              "I'm sorry",
              "Apologies",
              "My apologies",
              "I apologize"
        ],
          "responses": [
              "No worries! It's all good.",
              "It's okay! Don't worry about it.",
              "No problem, all forgiven.",
              "It's all right, don't worry."
        ]
      },
      {
          "tag": "affirmative",
          "patterns": [
              "Yes",
              "Yeah",
              "Of course",
              "Sure",
              "Definitely",
              "Absolutely"
        ],
          "responses": [
              "Great! How can I assist you further?",
              "Awesome! What else can I help you with?",
              "Thanks for confirming! What's next?",
              "Wonderful! What's on your mind?"
        ]
      },
      {
          "tag": "negative",
          "patterns": [
              "No",
              "Nope",
              "Not really",
              "I don't think so",
              "Definitely not",
              "Not at all"
        ],
          "responses": [
              "Okay, let me know if you change your mind.",
              "Alright, feel free to ask me anything else!",
              "No problem, let me know if you need anything.",
              "Got it! I'm here if you need anything later."
        ]
      },
      {
          "tag": "django_project_creation",
          "category": "Django",
          "patterns": [
              "What is a correct syntax for creating a Django project?",
              "How do I create a new Django project?",
              "What command starts a Django project?",
              "Syntax for starting a Django project?",
              "Command to initialize a Django project?",
              "How can I create a project using Django?",
              "Which command is used to set up a Django project?",
              "How to start a new Django project?",
              "Steps to create a Django project?",
              "How to initiate a Django project?",
              "Can you guide me to start a Django project?",
              "What's the command for starting a project in Django?",
              "What’s the right way to create a Django project?",
              "Instructions to set up a Django project?",
              "What steps are involved in creating a new Django project?",
              "How do you start a Django project?",
              "Can you show me how to create a Django project?",
              "What is the first step to set up a Django project?",
              "Starting a Django project: what command to use?",
              "What are the steps to create a Django project from scratch?",
              "How do I begin with a Django project?",
              "Which command should I use to create a new Django project?",
              "What command is needed to start a Django project?",
              "What is the correct command for setting up a Django project?"
          ],
          "responses": [
              "To start a new Django project, use the command: `django-admin startproject project_name`.",
              "Ensure you have Django installed before creating a project using `django-admin`.",
              "Use `django-admin startproject <project_name>` to initialize a Django project. Replace `<project_name>` with your desired project name."
          ]
      },
      {
          "tag": "django_runserver",
          "category": "Django",
          "patterns": [
              "What is the syntax to start a Django development server?",
              "How do I run a Django server?",
              "What command is used to start the server in Django?",
              "How can I start the Django server?",
              "How do you initiate the development server in Django?",
              "What do I type to run the server in Django?",
              "What command activates the Django local server?",
              "How to launch Django development server?",
              "Django command for running a local server?",
              "How do I start the Django runserver?",
              "Command for launching a Django server?",
              "How to start the development server in Django?",
              "Steps to initiate a Django server?",
              "What’s the way to run a server in Django?",
              "How do I launch the Django server?",
              "What's the command to run the Django development server?",
              "How do I start the server in Django?",
              "Can you tell me how to run a Django server?",
              "Steps to start the Django server?",
              "Which command starts the Django development server?",
              "How to initiate the Django server?",
              "What’s the command to start Django server?",
              "How to run the development server in Django?",
              "Can you explain how to start the Django server?",
              "What's needed to start the Django server?",
              "Instructions to start a server in Django?"
          ],
          "responses": [
              "To start the server, use the command: `python manage.py runserver`. Access the server at `http://127.0.0.1:8000`.",
              "Ensure you're in the project root directory and use the command `python manage.py runserver` to start the server.",
              "You can stop the server by pressing `Ctrl+C` in the terminal."
          ]
      },
      {
          "tag": "django_folder_content",
          "category": "Django",
          "patterns": [
              "Which file is not part of the Django project folder?",
              "What files are included in a new Django project?",
              "Which file is missing in a default Django project folder?",
              "How do I know what files are part of a Django project?",
              "Which file doesn't belong to a Django project's folder structure?",
              "What files are created when initializing a Django project?",
              "Is templates.py included in the default Django project?",
              "What is the default content of a Django project folder?",
              "Which files are typically found in a Django project?",
              "What files make up a basic Django project?",
              "What’s included in the default Django project structure?",
              "List of files in a new Django project?",
              "What’s the standard file structure in Django?",
              "What are the core files in a Django project?",
              "Which files are included in a default Django project?",
              "What files should I expect in a Django project folder?",
              "List the files in a new Django project.",
              "What is the default file structure for a Django project?",
              "What files are generated when creating a Django project?",
              "Which files come with a new Django project?",
              "What are the initial files in a Django project?",
              "Can you list the default files in a Django project?",
              "What are the standard files in a Django project?",
              "What files are part of a new Django project?",
              "What’s the default folder structure in Django?",
              "Which files are essential in a Django project?",
              "What are the basic files in a Django project?",
              "What files are present in a new Django project?"
          ],
          "responses": [
              "A Django project folder typically includes files like `manage.py`, `settings.py`, and `urls.py`. It does not include `templates.py` by default.",
              "The `templates` directory is created manually for organizing template files, rather than being a Python file.",
              "To explore the default files, navigate to the project folder after running `django-admin startproject`."
          ]
      },
      {
          "tag": "django_written_language",
          "category": "Django",
          "patterns": [
              "Which language is Django written in?",
              "What programming language does Django use?",
              "Django is written in what language?",
              "What language powers Django?",
              "In what language is the Django framework built?",
              "What is the base programming language for Django?",
              "Django is implemented using which language?",
              "What language is behind Django's framework?",
              "What language is Django written in?",
              "What programming language does Django use?",
              "Django is written in which language?",
              "What language is Django developed with?",
              "In which language is Django coded?",
              "Which programming language powers Django?",
              "What language is used for Django?",
              "Django is based on which language?",
              "What language does Django employ?",
              "Which programming language is Django built on?",
              "Django is implemented in which language?",
              "What language forms the basis of Django?",
              "Which language is used to develop Django?",
              "What coding language is Django written in?",
              "What language underpins Django?"
          ],
          "responses": [
              "Django is written in Python, a versatile language known for its readability and wide use in web development.",
              "The framework is based on Python, a popular language for scripting and building scalable web applications.",
              "Python powers Django, providing its clean syntax and rapid development capabilities."
          ]
      },
      {
          "tag": "django_handle_requests",
          "category": "Django",
          "patterns": [
              "In Django, incoming requests are handled in which file?",
              "What file manages requests in Django?",
              "Where are URL routing and request handling configured in Django?",
              "Which file handles HTTP requests in Django?",
              "Where can I define how requests are processed in Django?",
              "How does Django map requests to specific functions?",
              "Where do I define routing rules in Django?",
              "What file is used for request handling in Django?",
              "Which file in Django is responsible for URL routing?",
              "How are incoming requests managed in Django?",
              "What’s the main file for handling requests in Django?",
              "Where are request handlers configured in Django?",
              "File responsible for handling requests in Django?",
              "How does Django manage incoming HTTP requests?",
              "Which file handles requests in Django?",
              "Where are requests managed in Django?",
              "How are requests routed in Django?",
              "Which file is used to manage requests in Django?",
              "Where do I configure request handling in Django?",
              "How does Django process requests?",
              "Which file manages URL routing in Django?",
              "How are requests handled in Django?",
              "Where are request routes configured in Django?",
              "What file handles routing in Django?",
              "How does Django handle HTTP requests?",
              "In which file are requests handled in Django?"
          ],
          "responses": [
              "Incoming requests in Django are typically routed using the `urls.py` file.",
              "The `urls.py` file maps request paths to the corresponding views.",
              "For handling requests, define URL patterns in the `urls.py` file and link them to views."
          ]
      },
      {
          "tag": "django_admin_interface",
          "category": "Django",
          "patterns": [
              "How do I create a superuser in Django?",
              "What is the command to create an admin user in Django?",
              "How can I set up an admin account for Django's admin interface?",
              "What is the syntax for creating a superuser in Django?",
              "How do I access Django's admin panel as an admin?",
              "How can I generate a superuser for the Django admin?",
              "What command is used to make a superuser in Django?",
              "Steps to create a Django admin user?",
              "How to create an admin user in Django?",
              "Command for creating a superuser in Django?",
              "How do I set up a Django superuser?",
              "What is the process for creating a superuser in Django?",
              "Creating a superuser for Django: what command to use?",
              "How to configure an admin user in Django?",
              "What steps to follow to create a superuser in Django?",
              "How to set up an admin account in Django?",
              "Can you guide me to create a superuser in Django?",
              "What’s the way to create an admin user in Django?",
              "Procedure to generate a superuser in Django?",
              "How to create a superuser for Django's admin?"
          ],
          "responses": [
              "To create a superuser, run the command: `python manage.py createsuperuser`. Follow the prompts to set a username and password.",
              "Before running the command, ensure your database migrations are up-to-date with `python manage.py migrate`.",
              "Once the superuser is created, access the admin panel at `/admin` in your application."
          ]
      },
      {
          "tag": "general_django",
          "category": "Django",
          "patterns": [
              "Tell me about Django.",
              "What can I do with Django?",
              "Explain Django in simple terms.",
              "What is Django used for?",
              "How does Django help in web development?",
              "Why should I use Django?",
              "What makes Django unique?",
              "What are the benefits of using Django?",
              "Overview of Django framework?",
              "Django framework explained?",
              "What’s the purpose of Django?",
              "How can Django improve my web development?",
              "Why is Django popular?",
              "Django's key features?",
              "Why should I choose Django?",
              "What is Django good for?",
              "How does Django stand out?",
              "What can Django do for me?",
              "How can I benefit from using Django?",
              "What are the advantages of Django?",
              "How does Django simplify web development?",
              "What makes Django a good choice for web projects?",
              "Why use Django for web development?",
              "What are the key advantages of Django?",
              "What are the main features of Django?",
              "How can Django help me build web applications?",
              "Why is Django considered a powerful framework?",
              "What makes Django different from other frameworks?",
              "What are Django’s strengths?",
              "Why is Django preferred for web development?"
          ],
          "responses": [
              "Django is a high-level Python framework for building robust and scalable web applications.",
              "It simplifies web development by providing tools for database interaction, URL routing, and templating.",
              "Django is designed to help developers build secure and efficient web applications rapidly."
          ]
      }
   ]
}   
  